In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
! pip install opencv-python


In [ ]:
import threading
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import csv
from moviepy.editor import VideoFileClip

def preprocess_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = cv2.GaussianBlur(gray_image, (3, 3), 0)
    return gray_image

def abs_diff(pre_image, curr_image):
    gray_pre_image = preprocess_image(pre_image)
    gray_curr_image = preprocess_image(curr_image)
    diff = cv2.absdiff(gray_pre_image, gray_curr_image)
    res, diff = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    cnt_diff = np.sum(np.sum(diff))
    return cnt_diff

def exponential_smoothing(alpha, s):
    s_temp = [s[0]]
    for i in range(1, len(s), 1):
        s_temp.append(alpha * s[i - 1] + (1 - alpha) * s_temp[i - 1])
    return s_temp

class KeyFrameGetter:
    def __init__(self, video_path=None, output_path=None, window=25):
        self.window = window
        self.video_path = video_path
        self.output_path = output_path
        self.diff = []
        self.idx = []

    def load_diff_between_frm(self, smooth=False, alpha=0.00):
        cap = cv2.VideoCapture(self.video_path)
        diff = []
        frm = 0
        pre_image = np.array([])
        curr_image = np.array([])

        while True:
            frm = frm + 1
            success, data = cap.read()
            if not success:
                break

            if frm == 1:
                pre_image = data
                curr_image = data
            else:
                pre_image = curr_image
                curr_image = data

            diff.append(abs_diff(pre_image, curr_image))

            if frm % 100 == 0:
                print('Detect Frame:', str(frm))
        cap.release()

        if smooth:
            diff = exponential_smoothing(alpha, diff)

        self.diff = np.array(diff)
        mean = np.mean(self.diff)
        dev = np.std(self.diff)
        self.diff = (self.diff - mean) / dev

        self.pick_idx()
        return

    def pick_idx(self):
        for i, d in enumerate(self.diff):
            ub = len(self.diff) - 1
            lb = 0
            if not i - self.window // 2 < lb:
                lb = i - self.window // 2
            if not i + self.window // 2 > ub:
                ub = i + self.window // 2

            comp_window = self.diff[lb:ub]
            if d >= max(comp_window):
                self.idx.append(i)

        tmp = np.array(self.idx)
        tmp = tmp + 1
        self.idx = tmp.tolist()
        print("Extract the Frame Index:" + str(self.idx))

    def save_key_frame(self, interval=2):
        video_name = os.path.splitext(os.path.basename(self.video_path))[0]
        video_output_path = os.path.join(self.output_path, video_name)
        if not os.path.exists(video_output_path):
            os.makedirs(video_output_path)

        cap = cv2.VideoCapture(self.video_path)
        frm = 0
        idx = set(self.idx)
        key_frames_count = 0
        interval_frames_count = 0
        interval_start_time = -interval

        while True:
            frm = frm + 1
            success, data = cap.read()
            if not success:
                break

            current_time = frm / cap.get(cv2.CAP_PROP_FPS)
            if current_time - interval_start_time >= interval:
                interval_start_time = current_time
                interval_frames_count += 1

                # Create a new folder for key frames within the current interval
                interval_subfolder = os.path.join(video_output_path, f"interval_{interval_frames_count}")
                if not os.path.exists(interval_subfolder):
                    os.makedirs(interval_subfolder)

                # Create a folder for all frames within this interval
                all_frames_folder = os.path.join(interval_subfolder, "All_frames")
                os.makedirs(all_frames_folder)

                print(f"Number of key frames in {interval} seconds: {key_frames_count}")
                key_frames_count = 0

            if frm in idx:
                # Save the identified key frame
                saved_img_nm = os.path.join(interval_subfolder, f"Keyframe_{frm}.png")
                cv2.imwrite(saved_img_nm, data)
                idx.remove(frm)

            # Save all frames in the "All_frames" folder within the current interval
            saved_all_frame_nm = os.path.join(all_frames_folder, f"{frm}.png")
            cv2.imwrite(saved_all_frame_nm, data)

            if not idx:
                print('Done!')
                break

    def extract_all_frames(self, output_folder, interval=2):
        cap = cv2.VideoCapture(self.video_path)
        frm = 0
        interval_frames_count = 0
        interval_start_time = -interval

        while True:
            frm = frm + 1
            success, data = cap.read()
            if not success:
                break

            current_time = frm / cap.get(cv2.CAP_PROP_FPS)
            if current_time - interval_start_time >= interval:
                interval_start_time = current_time
                interval_frames_count += 1

                interval_subfolder = os.path.join(output_folder, f"interval_{interval_frames_count}")
                if not os.path.exists(interval_subfolder):
                    os.makedirs(interval_subfolder)
                print(f"Saved frame {interval_frames_count} at {current_time} seconds")

                saved_img_nm = os.path.join(interval_subfolder , f"{frm}.png")
                cv2.imwrite(saved_img_nm, data)

                video_clip = VideoFileClip(self.video_path)
                start_time = max(0, (interval_frames_count - 1) * interval)
                end_time = min(video_clip.duration, interval_frames_count * interval)

                subclip = video_clip.subclip(start_time, end_time)

                mp4_output_path = os.path.join(interval_subfolder, f"interval_{interval_frames_count}.mp4")
                wav_output_path = os.path.join(interval_subfolder, f"interval_{interval_frames_count}.wav")

                subclip.write_videofile(mp4_output_path, codec='libx264', audio_codec='aac')
                subclip.audio.write_audiofile(wav_output_path, codec='pcm_s16le')

        cap.release()
        print('Done!')

    def plot_diff_time(self):
        plt.plot(self.diff, '-b')
        plt.plot(np.array(self.idx) - 1, [self.diff[i] for i in self.idx], 'or')
        plt.xlabel('Frame Pair Index')
        plt.ylabel('Difference')
        plt.legend(['Each Frame', 'Extract Frame'])
        plt.title("The Difference for Each Pair of Frame")
        plt.plot()
        plt.show()

    def save_key_frame_info_to_csv(self, csv_path, interval=2):
        with open(csv_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Frame Counts', 'Key Frame Index', 'Key Frame Path', 'Key Frame Image', 'MP4 Path', 'WAV Path'])

            for idx, key_frame in enumerate(self.idx):
                key_frame_path = f"interval_{idx + 1}/keyframe_{key_frame}.png"
                key_frame_image = f"key_frame_{key_frame}.png"

                mp4_path = f"interval_{idx + 1}/interval_{idx + 1}.mp4"
                wav_path = f"interval_{idx + 1}/interval_{idx + 1}.wav"

                for root, dirs, files in os.walk(self.output_path):
                    if key_frame_path in files:
                        key_frame_path = os.path.join(root, key_frame_path)
                        break

                for root, dirs, files in os.walk(self.output_path):
                    if key_frame_image in files:
                        key_frame_image = os.path.join(root, key_frame_image)
                        break

                for root, dirs, files in os.walk(self.output_path):
                    if mp4_path in files:
                        mp4_path = os.path.join(root, mp4_path)
                        break

                for root, dirs, files in os.walk(self.output_path):
                    if wav_path in files:
                        wav_path = os.path.join(root, wav_path)
                        break

                writer.writerow([idx + 1, key_frame, key_frame_path, key_frame_image, mp4_path, wav_path])

        print(f"Key frame information saved to {csv_path}")

class ProcessVideoThread(threading.Thread):
    def __init__(self, video_path, output_dir, window_size, all_video_keyframes, lock):
        threading.Thread.__init__(self)
        self.video_path = video_path
        self.output_dir = output_dir
        self.window_size = window_size
        self.all_video_keyframes = all_video_keyframes
        self.lock = lock  # Add a lock to ensure proper synchronization

    def run(self):
        framegen = KeyFrameGetter(self.video_path, self.output_dir, self.window_size)
        framegen.load_diff_between_frm(alpha=0.07)
        framegen.save_key_frame(interval=2)

        with self.lock:  # Acquire the lock before updating shared data
            for idx, key_frame in enumerate(framegen.idx):
                key_frame_path = f"interval_{idx + 1}/key_frames_{key_frame}.png"
                key_frame_image = f"keyframe_{key_frame}.png"
                colab_video_path = os.path.join(src_folder, video_file)

                mp4_path = f"interval_{idx + 1}/interval_{idx + 1}.mp4"
                wav_path = f"interval_{idx + 1}/interval_{idx + 1}.wav"

                self.all_video_keyframes.append([video_file, idx + 1, key_frame, key_frame_path, key_frame_image, colab_video_path, mp4_path, wav_path])

        output_folder_frames = os.path.join(self.output_dir, f"{os.path.splitext(video_file)[0]}")

        if not os.path.exists(output_folder_frames):
            os.makedirs(output_folder_frames)

        framegen.extract_all_frames(output_folder_frames, interval=2)

if __name__ == '__main__':
    src_folder = "/content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_videos"
    output_dir = "/content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes"
    window_size = 100
    all_video_keyframes = []

    lock = threading.Lock()  # Create a lock for synchronization

    num_threads = 5  # Specify the number of threads
    threads = []

    if os.path.exists(src_folder) and os.path.isdir(src_folder):
        video_files = [file for file in os.listdir(src_folder) if file.endswith(".mp4")]

        for video_file in video_files:
            video_path = os.path.join(src_folder, video_file)

            # Create and start threads
            for _ in range(num_threads):
                thread = ProcessVideoThread(video_path, output_dir, window_size, all_video_keyframes, lock)
                thread.start()
                threads.append(thread)

            # Wait for all threads to finish
            for thread in threads:
                thread.join()

        all_video_csv_path = os.path.join(output_dir, "Final_Input.csv")
        with open(all_video_csv_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Video File', 'Frame Counts', 'Key Frame Index', 'Key Frame Path', 'Key Frame Image', 'Video Path', 'MP4 Path', 'WAV Path'])
            writer.writerows(all_video_keyframes)

        print("All videos processed. Key frame information for all videos saved to 'Final_Input.csv'")
    else:
        print(f"The source folder '{src_folder}' does not exist or is not a directory.")


Detect Frame: 100
Detect Frame: 100
Detect Frame: 100
Detect Frame: 100
Detect Frame: 100
Detect Frame: 200
Detect Frame: 200
Detect Frame: 200
Detect Frame: 200
Detect Frame: 200
Detect Frame: 300
Detect Frame: 300
Detect Frame: 300
Detect Frame: 300
Detect Frame: 300
Detect Frame: 400
Detect Frame: 400
Detect Frame: 400
Detect Frame: 400
Detect Frame: 400
Detect Frame: 500
Detect Frame: 500
Detect Frame: 500
Detect Frame: 500
Detect Frame: 500
Detect Frame: 600
Detect Frame: 600
Detect Frame: 600
Detect Frame: 600
Detect Frame: 600
Detect Frame: 700
Detect Frame: 700
Detect Frame: 700
Detect Frame: 700
Detect Frame: 700
Detect Frame: 800
Detect Frame: 800
Detect Frame: 800
Detect Frame: 800
Detect Frame: 800
Detect Frame: 900
Detect Frame: 900
Detect Frame: 900
Detect Frame: 900
Detect Frame: 900
Detect Frame: 1000
Detect Frame: 1000
Detect Frame: 1000
Detect Frame: 1000
Detect Frame: 1000
Detect Frame: 1100
Detect Frame: 1100
Detect Frame: 1100
Detect Frame: 1100
Detect Frame: 1100


Exception in thread Thread-17:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "<ipython-input-8-82b48a353899>", line 240, in run
  File "<ipython-input-8-82b48a353899>", line 123, in save_key_frame
  File "/usr/lib/python3.10/os.py", line 225, in makedirs
    mkdir(name, mode)
FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes/మీ ప్రొఫైల్ వెరిఫై చేసుకోవడం ఎలా  Indus EasyWheels/interval_1/All_frames'


Extract the Frame Index:[12, 129, 257, 322, 385, 456, 513, 641, 769, 886, 1025, 1088, 1153, 1281, 1409, 1537, 1609, 1665, 1793, 1910, 1970]
Detect Frame: 2000


Exception in thread Thread-19:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "<ipython-input-8-82b48a353899>", line 240, in run
  File "<ipython-input-8-82b48a353899>", line 123, in save_key_frame
  File "/usr/lib/python3.10/os.py", line 225, in makedirs
    mkdir(name, mode)
FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes/మీ ప్రొఫైల్ వెరిఫై చేసుకోవడం ఎలా  Indus EasyWheels/interval_1/All_frames'


Extract the Frame Index:[12, 129, 257, 322, 385, 456, 513, 641, 769, 886, 1025, 1088, 1153, 1281, 1409, 1537, 1609, 1665, 1793, 1910, 1970]
Detect Frame: 2000
Extract the Frame Index:[12, 129, 257, 322, 385, 456, 513, 641, 769, 886, 1025, 1088, 1153, 1281, 1409, 1537, 1609, 1665, 1793, 1910, 1970]


...

Saved frame 80 at 158.16 seconds
Moviepy - Building video /content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes/Powering 67 villages with Solar Powered Street Lights An IndusInd Bank CSR Initiative/interval_80/interval_80.mp4.
MoviePy - Writing audio in interval_80TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes/Powering 67 villages with Solar Powered Street Lights An IndusInd Bank CSR Initiative/interval_80/interval_80.mp4



t:  98%|█████████▊| 40/41 [00:01<00:00, 38.13it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_videos/Powering 67 villages with Solar Powered Street Lights An IndusInd Bank CSR Initiative.mp4, 2764800 bytes wanted but 0 bytes read,at frame 3990/3991, at time 159.60/159.61 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes/Powering 67 villages with Solar Powered Street Lights An IndusInd Bank CSR Initiative/interval_80/interval_80.mp4
MoviePy - Writing audio in /content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes/Powering 67 villages with Solar Powered Street Lights An IndusInd Bank CSR Initiative/interval_80/interval_80.wav


MoviePy - Done.
Done!
All videos processed. Key frame information for all videos saved to 'Final_Input.csv'


In [3]:
import os
from moviepy.video.io.VideoFileClip import VideoFileClip
import shutil
import pandas as pd

def convert_video_to_audio(video_path, audio_path):
    video_clip = VideoFileClip(video_path)
    video_clip.audio.write_audiofile(audio_path)
    video_clip.close()

def process_videos(input_folder, output_folder):
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith(".mp4"):
                video_path = os.path.join(root, file)
                audio_path = os.path.join(output_folder, file.replace(".mp4", ".wav"))

                # Convert MP4 to MP3
                convert_video_to_audio(video_path, audio_path)

                # Move both MP4 and MP3 to the corresponding folder in the output directory
                folder_name = file.replace(".mp4", "")
                destination_folder = os.path.join(output_folder, folder_name)

                if not os.path.exists(destination_folder):
                    os.makedirs(destination_folder)

                destination_path_mp4 = os.path.join(destination_folder, file)
                destination_path_mp3 = os.path.join(destination_folder, file.replace(".mp4", ".wav"))

                shutil.move(video_path, destination_path_mp4)
                shutil.move(audio_path, destination_path_mp3)

if __name__ == "__main__":
    input_folder = "/content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_videos"                          # Input_Folder
    output_folder = "/content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes"                # Output_Folder

    process_videos(input_folder, output_folder)


MoviePy - Writing audio in /content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes/మీ ప్రొఫైల్ వెరిఫై చేసుకోవడం ఎలా  Indus EasyWheels.wav


MoviePy - Done.
MoviePy - Writing audio in /content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes/ਸਾਡੇ ਨਾਲ ਸੰਪਰਕ ਕਿਵੇਂ ਕਰੀਏ  Indus EasyWheels.wav


MoviePy - Done.
MoviePy - Writing audio in /content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes/ਵਾਹਨ ਸੂਚੀਆਂ ਰਾਹੀਂ ਬ੍ਰਾਊਜ਼ ਕਿਵੇਂ ਕਰੀਏ  Indus EasyWheels.wav


MoviePy - Done.
MoviePy - Writing audio in /content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes/वॉचलिस्ट कार्यक्षमता का उपयोग कैसे करें  Indus EasyWheels.wav


MoviePy - Done.
MoviePy - Writing audio in /content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes/ఇండస్ఇండ్ బ్యాంక్ సేవింగ్స్ అకౌంట్ తో మీకు నచ్చిన ఒక అకౌంట్ నంబర్ ఎంచుకోండి!.wav


...

MoviePy - Done.
MoviePy - Writing audio in /content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes/ਵਾਚਲਿਸਟ ਫੰਕਸ਼ਨੈਲਿਟੀ ਦੀ ਵਰਤੋਂ ਕਿਵੇਂ ਕਰੀਏ  Indus EasyWheels.wav


MoviePy - Done.
MoviePy - Writing audio in /content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes/ਕਿਵੇਂ ਰਜਿਸਟਰ ਕਰਨਾ ਹੈ  Indus EasyWheels.wav


MoviePy - Done.
MoviePy - Writing audio in /content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes/अपनी प्रोफाइल को कैसे सत्यापित करें  Indus EasyWheels.wav


MoviePy - Done.
MoviePy - Writing audio in /content/drive/MyDrive/Akash_Ads_keyframes/Indus_bank_keyframes/Powering 67 villages with Solar Powered Street Lights An IndusInd Bank CSR Initiative.wav


MoviePy - Done.


In [ ]:
import os
import shutil

def merge_duplicate_folders(parent_directory):
    # Get a list of all subdirectories in the parent directory
    subdirectories = [d for d in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, d))]

    # Create a dictionary to store base folder names and their corresponding paths
    folder_dict = {}

    # Iterate through subdirectories
    for folder in subdirectories:
        folder_path = os.path.join(parent_directory, folder)

        # Extract the base folder name without the duplicate suffix
        base_folder_name = extract_base_name(folder)

        # Check if the base folder name already exists in the dictionary
        if base_folder_name in folder_dict:
            # Merge the contents of the current folder with the existing one
            merge_folders(folder_path, folder_dict[base_folder_name])
        else:
            # Add the folder to the dictionary
            folder_dict[base_folder_name] = folder_path

def merge_folders(source_folder, destination_folder):
    # Ensure the destination folder exists
    os.makedirs(destination_folder, exist_ok=True)

    # Move the contents of the source folder to the destination folder
    for item in os.listdir(source_folder):
        source_item = os.path.join(source_folder, item)
        destination_item = os.path.join(destination_folder, item)

        # If the item is a directory, recursively merge its contents
        if os.path.isdir(source_item):
            merge_folders(source_item, destination_item)
        else:
            # If the item is a file, move it to the destination folder
            handle_duplicate_files(source_item, destination_item)

    # Remove the now-empty source folder
    os.rmdir(source_folder)

def handle_duplicate_files(source_file, destination_file):
    # Handle the case where the destination file already exists
    base_name, extension = os.path.splitext(destination_file)
    count = 1

    while os.path.exists(destination_file):
        new_name = f"{base_name}_{count}{extension}"
        destination_file = os.path.join(os.path.dirname(destination_file), new_name)
        count += 1

    shutil.move(source_file, destination_file)

def extract_base_name(folder_name):
    # Remove the duplicate suffix, if present
    return folder_name.split(' (')[0].strip()

# Example usage:
parent_directory = "/content/drive/MyDrive/Akash_Ads_keyframes/Akash/Rapido/Rapido_keyframes"
merge_duplicate_folders(parent_directory)
